In [17]:
import torch
import torch.nn as nn
import sys
def generate(X, alpha, beta, eta, gamma, estimator="LATE"):
    phi = torch.sigmoid(X@beta)
    c1, c2, c3, c4 = phi[:,0], phi[:,1], phi[:,2], phi[:,3]
    if estimator == 'LATE':
        c0 = torch.tanh(X @ alpha)
    elif estimator == 'MLATE':
        c0 = torch.exp(X @ alpha)
    c5 = torch.exp(X@eta)
    if estimator == 'LATE':
        f0 = (c5*(2-c0)+c0-torch.sqrt(c0**2 * (c5-1)**2 + 4*c5)) / (2 * (c5-1))
        f1 = f0 + c0
    elif estimator == 'MLATE':
        f0 = (-(c0+1)*c5+torch.sqrt(c5**2*(c0-1)**2 + 4*c0*c5)) / (2*c0*(1-c5))
        f1 = f0 * c0
    p011 = (1-c1)*(1-c2)*c3
    p001 = (1-c1)*(1-c2)*(1-c3)
    p110 = (1-c1)*c2*c4
    p100 = (1-c1)*c2*(1-c4)
    p111 = f1*c1 + p110
    p010 = f0*c1 + p011
    p101 = 1-p001-p011-p111
    p000 = 1-p010-p100-p110
    VIden = torch.sigmoid(X @ gamma)

    Z = torch.bernoulli(VIden)

    p1 = torch.column_stack((p001, p101, p011, p111))
    p0 = torch.column_stack((p000, p100, p010, p110))
    p = Z.unsqueeze(1) * p1 + (1-Z).unsqueeze(1) * p0
    idx = torch.multinomial(p, num_samples=1).squeeze(1)
    cand = torch.LongTensor([[0,0], [1,0], [0,1], [1,1]])
    DY = cand[idx]
    return Z, DY[:,0], DY[:,1]

def nll(X,Z,D,Y,alpha,beta,eta,gamma,estimator='LATE'):
    phi = torch.sigmoid(X@beta)
    c1, c2, c3, c4 = phi[:,0], phi[:,1], phi[:,2], phi[:,3]
    if estimator == 'LATE':
        c0 = torch.tanh(X @ alpha)
    elif estimator == 'MLATE':
        c0 = torch.exp(X @ alpha)
    c5 = torch.exp(X@eta)
    if estimator == 'LATE':
        f0 = (c5*(2-c0)+c0-torch.sqrt(c0**2 * (c5-1)**2 + 4*c5)) / (2 * (c5-1))
        f1 = f0 + c0
    elif estimator == 'MLATE':
        f0 = (-(c0+1)*c5+torch.sqrt(c5**2*(c0-1)**2 + 4*c0*c5)) / (2*c0*(1-c5))
        f1 = f0 * c0
    p011 = (1-c1)*(1-c2)*c3
    p001 = (1-c1)*(1-c2)*(1-c3)
    p110 = (1-c1)*c2*c4
    p100 = (1-c1)*c2*(1-c4)
    p111 = f1*c1 + p110
    p010 = f0*c1 + p011
    p101 = 1-p001-p011-p111
    p000 = 1-p010-p100-p110

    d = torch.sigmoid(X@gamma)
    l = D*Y*Z*p111*d + (1-D)*Y*Z*p011*d + D*(1-Y)*Z*p101*d + (1-D)*(1-Y)*Z*p001*d + D*Y*(1-Z)*p110*(1-d) + (1-D)*Y*(1-Z)*p010*(1-d) + D*(1-Y)*(1-Z)*p100*(1-d) + (1-D)*(1-Y)*(1-Z)*p000*(1-d)
    return torch.mean(torch.log(l.clamp(1e-10)))

def square_loss(X, Z, D, Y, alpha, beta, gamma, eta, estimator, strategy='identity'):
    d = torch.sigmoid(X@gamma)
    phi = torch.sigmoid(X@beta)
    phi1, phi2, phi3, phi4 = phi[:,0], phi[:,1], phi[:,2], phi[:,3]
    OP = torch.exp(X@eta)
    f = (d**Z) * ((1-d)**(1-Z))
    if estimator == 'LATE':
        theta = torch.tanh(X@alpha)
        H = Y - D * theta
        f0 = (OP*(2-theta)+theta-torch.sqrt(theta**2*(OP-1)**2+4*OP))/(2*(OP-1))
        E = f0*phi1 + (1-phi1)*(1-phi2)*phi3 + (1-phi1)*phi2*phi4 - theta*(1-phi1)*phi2
    elif estimator == 'MLATE':
        theta = torch.exp(X@alpha)
        H = Y * theta**(-D)
        f0 = (-(theta+1)*OP+torch.sqrt(OP**2*(theta-1)**2+4*theta*OP)) / (2*theta*(1-OP))
        E = f0*phi1 + (1-phi1)*phi2*phi4/theta + (1-phi1)*(1-phi2)*phi3
    
    if strategy == 'identity':
        return (torch.sum((2*Z-1)*(H-E)/f))**2

def MLE(X, estimator='LATE', dr=True):
    alpha0 = torch.tensor([0.0, -1.0])
    beta0 = (torch.ones(size=(4,2)) * torch.tensor([-0.4,0.8])).T
    eta0 = torch.tensor([-0.4, 1.0])
    gamma0 = torch.tensor([0.1, -1.0])
    Z, D, Y = generate(X, alpha0, beta0, eta0, gamma0, estimator)
    minimum = (-nll(X,Z,D,Y,alpha0,beta0,eta0,gamma0, estimator)).item()
    # print('minimum', minimum)
    # alpha = nn.Parameter(torch.tensor([0.5, 0.5]))
    # beta = nn.Parameter((torch.ones(size=(4,2)) * torch.tensor([-0.5,0.5])).T)
    # eta = nn.Parameter(torch.tensor([-0.5, 0.5]))
    alpha = nn.Parameter(torch.rand(size=(2,))*2-1)
    beta = nn.Parameter(torch.rand(size=(2,4))*2-1)
    eta = nn.Parameter(torch.rand(size=(2,))*2-1)
    gamma = nn.Parameter(torch.rand(size=(2,))*2-1)
    opt = torch.optim.Adam(params=(alpha, beta, eta, gamma), lr=1e-3, weight_decay=0)
    optloss = float('inf')
    for i in range(5000):
        loss = -nll(X,Z,D,Y,alpha,beta,eta,gamma, estimator)
        if loss.item() < optloss:
            opt.zero_grad()
            optloss = loss.item()
            mlealpha = alpha.detach().clone()
            mlebeta = beta.detach().clone()
            mleeta = eta.detach().clone()
            mlegamma = gamma.detach().clone()
        # print('Iter {}\t | loss {:.04f}'.format(i+1, loss.item()))
        loss.backward()
        opt.step()

    if not dr:
        return mlealpha, mlebeta, mleeta, mlegamma, minimum, optloss

    alpha = nn.Parameter(mlealpha,requires_grad=True)
    opt = torch.optim.Adam(params=(alpha,), lr=1e-3, weight_decay=0)
    optloss = float('inf')
    for i in range(200):
        opt.zero_grad()
        sq_loss = square_loss(X, Z, D, Y, alpha, mlebeta, mlegamma, mleeta, estimator)
        print('Iter {}\t | sq_loss {:.04f}'.format(i+1, sq_loss.item()))
        if sq_loss.item() < optloss:
            optloss = sq_loss.item()
            dralpha = alpha.detach().clone()
        sq_loss.backward()
        opt.step()
    return mlealpha, dralpha, mlebeta, mleeta, mlegamma, minimum, optloss

def mle_bth_L():
    N = 1000
    NR = 1000
    torch.manual_seed(24)
    mlealphas = torch.zeros(size=(NR, 2))
    dralphas = torch.zeros(size=(NR, 2))
    minimums, optlosses = [], []
    X = torch.column_stack((torch.ones(N)*1.0, torch.rand(N)*2-1))
    for i in range(NR):
        mlealpha, dralpha, mlebeta, mleeta, mlegamma, minimum, optloss = MLE(X)
        print('{} Experiement | Difference {:.04f} | MLEAlpha: ({:.04f}, {:.04f})'.format(i+1, optloss-minimum, mlealpha[0].item(), mlealpha[1].item()))
        print('{} Experiement | Difference {:.04f} | drAlpha: ({:.04f}, {:.04f})'.format(i+1, optloss-minimum, dralpha[0].item(), dralpha[1].item()))
        mlealphas[i] = mlealpha
        dralphas[i] = dralpha
        minimums.append(minimum)
        optlosses.append(optloss)
    torch.save(mlealphas, 'mle_bth_L')
    torch.save(dralphas, 'dru_bth_L')
## performs well

####### mle.bth 
def mle_bth_M():
    N = 1000
    NR = 1000
    torch.manual_seed(24)
    mlealphas = torch.zeros(size=(NR, 2))
    dralphas = torch.zeros(size=(NR, 2))
    minimums, optlosses = [], []
    X = torch.column_stack((torch.ones(N)*1.0, torch.rand(N)*2-1))
    for i in range(NR):
        mlealpha, dralpha, mlebeta, mleeta, mlegamma, minimum, optloss = MLE(X, estimator='MLATE')
        print('{} Experiement | Difference {:.04f} | MLEAlpha: ({:.04f}, {:.04f})'.format(i+1, optloss-minimum, mlealpha[0].item(), mlealpha[1].item()))
        print('{} Experiement | Difference {:.04f} | drAlpha: ({:.04f}, {:.04f})'.format(i+1, optloss-minimum, dralpha[0].item(), dralpha[1].item()))
        mlealphas[i] = mlealpha
        dralphas[i] = dralpha
        minimums.append(minimum)
        optlosses.append(optloss)
    torch.save(mlealphas, 'mle_bth_M')
    torch.save(dralphas, 'dru_bth_M')

## performs well


###
##对初值敏感(没有先验知识时可能造成无法训练出来)，对学习率敏感
##添加贝叶斯先验分布时可能的改进


In [18]:
mle_bth_M()

Iter 1	 | sq_loss 599.3785
Iter 2	 | sq_loss 582.3530
Iter 3	 | sq_loss 565.6077
Iter 4	 | sq_loss 549.1438
Iter 5	 | sq_loss 532.9667
Iter 6	 | sq_loss 517.0794
Iter 7	 | sq_loss 501.4817
Iter 8	 | sq_loss 486.1799
Iter 9	 | sq_loss 471.1725
Iter 10	 | sq_loss 456.4640
Iter 11	 | sq_loss 442.0525
Iter 12	 | sq_loss 427.9427
Iter 13	 | sq_loss 414.1324
Iter 14	 | sq_loss 400.6237
Iter 15	 | sq_loss 387.4162
Iter 16	 | sq_loss 374.5091
Iter 17	 | sq_loss 361.9018
Iter 18	 | sq_loss 349.5952
Iter 19	 | sq_loss 337.5854
Iter 20	 | sq_loss 325.8719
Iter 21	 | sq_loss 314.4553
Iter 22	 | sq_loss 303.3297
Iter 23	 | sq_loss 292.4951
Iter 24	 | sq_loss 281.9486
Iter 25	 | sq_loss 271.6875
Iter 26	 | sq_loss 261.7065
Iter 27	 | sq_loss 252.0068
Iter 28	 | sq_loss 242.5820
Iter 29	 | sq_loss 233.4282
Iter 30	 | sq_loss 224.5429
Iter 31	 | sq_loss 215.9212
Iter 32	 | sq_loss 207.5592
Iter 33	 | sq_loss 199.4525
Iter 34	 | sq_loss 191.5970
Iter 35	 | sq_loss 183.9885
Iter 36	 | sq_loss 176.6213
I

KeyboardInterrupt: 